## Primeira versão Auto-pytorch - entendimento da biblioteca
- https://github.com/automl/Auto-PyTorch
- https://www.automl.org/automl-for-x/tabular-data/autopytorch/
- https://automl.github.io/Auto-PyTorch/development/
- https://automl.github.io/Auto-PyTorch/development/manual.html

# Esse notebook foi rodado dentro do .devcontainer para a imagem automlorg/autopytorch:v0.2 que foi pega do docker hub

In [14]:
#  Precisei instalar isso para conseguir rodar a TabularClassificationTask
! apt-get update

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [4,276 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates/multiverse amd64 Packages [34.6 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,589 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [30.9 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [4,187 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,297 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [4,663 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [4,463 kB]
Get:13 http:

In [15]:
!apt-get install libgl1-mesa-glx -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libbsd0 libdrm-amdgpu1 libdrm-common libdrm-intel1 libdrm-nouveau2
  libdrm-radeon1 libdrm2 libedit2 libelf1 libgl1 libgl1-mesa-dri libglapi-mesa
  libglvnd0 libglx-mesa0 libglx0 libllvm12 libpciaccess0 libsensors-config
  libsensors5 libvulkan1 libwayland-client0 libx11-6 libx11-data libx11-xcb1
  libxau6 libxcb-dri2-0 libxcb-dri3-0 libxcb-glx0 libxcb-present0
  libxcb-randr0 libxcb-shm0 libxcb-sync1 libxcb-xfixes0 libxcb1 libxdmcp6
  libxext6 libxfixes3 libxshmfence1 libxxf86vm1 mesa-vulkan-drivers
Suggested packages:
  pciutils lm-sensors
The following NEW packages will be installed:
  libbsd0 libdrm-amdgpu1 libdrm-common libdrm-intel1 libdrm-nouveau2
  libdrm-radeon1 libdrm2 libedit2 libelf1 libgl1 libgl1-mesa-dri
  libgl1-mesa-glx libglapi-mesa libglvnd0 libglx-mesa0 libglx0 libllvm12
  libpciaccess0 libsensors-config libsensors5 li

In [19]:
! apt-get install libglib2.0-0 -y
# dpkg --configure -a

In [ ]:
# Opcional
# apt-get install htop

## Tentativa 1

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../../auto-pytorch') 

In [3]:
from autoPyTorch.api.tabular_classification import TabularClassificationTask

In [21]:
# data and metric imports
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics


In [22]:
X, y = sklearn.datasets.load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = \
        sklearn.model_selection.train_test_split(X, y, random_state=1)

In [23]:
# initialise Auto-PyTorch api
api = TabularClassificationTask()

In [10]:
# help(TabularClassificationTask)

In [24]:
# Search for an ensemble of machine learning algorithms
api.search(
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    optimize_metric='accuracy',
    total_walltime_limit=300,
    func_eval_time_limit_secs=50
)

cannot allocate memory for thread-local data: ABORT
cannot allocate memory for thread-local data: ABORT
cannot allocate memory for thread-local data: ABORT


[ERROR] [2025-01-26 16:41:49,347:Client-AutoPyTorch:6e40915e-dc04-11ef-8d2f-0242ac110002:1] Traditional prediction for lgb failed with run state StatusType.CRASHED.
Additional info:
error: Result queue is empty
exit_status: <class 'pynisher.limit_function_call.AnythingException'>
subprocess_stdout: 
subprocess_stderr: 
exitcode: 127
configuration_origin: traditional
[ERROR] [2025-01-26 16:42:49,135:Client-AutoPyTorch:6e40915e-dc04-11ef-8d2f-0242ac110002:1] Traditional prediction for catboost failed with run state StatusType.TIMEOUT.
Additional info:
error: Timeout
configuration_origin: traditional


OpenBLAS blas_thread_init: pthread_create failed for thread 16 of 32: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC -1 current, -1 max
OpenBLAS blas_thread_init: pthread_create failed for thread 17 of 32: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC -1 current, -1 max
OpenBLAS blas_thread_init: pthread_create failed for thread 18 of 32: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC -1 current, -1 max
OpenBLAS blas_thread_init: pthread_create failed for thread 19 of 32: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC -1 current, -1 max
OpenBLAS blas_thread_init: pthread_create failed for thread 20 of 32: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC -1 current, -1 max
OpenBLAS blas_thread_init: pthread_create failed for thread 21 of 32: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC -1 current, -1 max
OpenBLAS blas_thread_init: p

[ERROR] [2025-01-26 16:43:45,402:Client-AutoPyTorch:6e40915e-dc04-11ef-8d2f-0242ac110002:1] Traditional prediction for knn failed with run state StatusType.TIMEOUT.
Additional info:
error: Timeout
configuration_origin: traditional


/usr/local/lib/python3.8/dist-packages/smac/intensification/parallel_scheduling.py:154: UserWarning: Hyperband is executed with 1 workers only. Consider to use pynisher to use all available workers.
  warnings.warn(
cannot allocate memory for thread-local data: ABORT
cannot allocate memory for thread-local data: ABORT
cannot allocate memory for thread-local data: ABORT
cannot allocate memory for thread-local data: ABORT
cannot allocate memory for thread-local data: ABORT
OpenBLAS blas_thread_init: pthread_create failed for thread 16 of 32: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC -1 current, -1 max
OpenBLAS blas_thread_init: pthread_create failed for thread 17 of 32: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC -1 current, -1 max
OpenBLAS blas_thread_init: pthread_create failed for thread 18 of 32: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC -1 current, -1 max
OpenBLAS blas_thread_init: pthread_cre

In [25]:
# Calculate test accuracy
y_pred = api.predict(X_test)
score = api.score(y_pred, y_test)
print("Accuracy score", score)

Accuracy score {'accuracy': 0.9844444444444445}


## Tentativa 2

In [3]:
import os
import tempfile as tmp
import warnings

from autoPyTorch.datasets.resampling_strategy import CrossValTypes

os.environ['JOBLIB_TEMP_FOLDER'] = tmp.gettempdir()
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

import sklearn.datasets
import sklearn.model_selection

from autoPyTorch.api.tabular_classification import TabularClassificationTask

In [4]:
import pandas as pd
from glob import glob

In [5]:
pd.set_option('display.max_columns', None)

In [6]:
csv_files = glob("../Data/data_Q4_2023/*/*csv", recursive=True)
# df = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)

In [7]:
columns_to_drop = ["date", "datacenter", "cluster_id", "vault_id",	"pod_id"]
columns_to_set = ["serial_number"]

In [10]:
df = pd.read_csv(csv_files[0])
df = df.drop(columns=columns_to_drop)

In [11]:
index_to_set_falure = list(range(10_000)) 
df.loc[index_to_set_falure, "failure"] = 1

df = df.set_index(columns_to_set)


In [12]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    df.drop(columns=["failure"]),
    df[["failure"]],
    random_state=12_01_2001,
    shuffle=True,
    stratify=df["failure"],
    test_size=0.2
)

In [13]:
api = TabularClassificationTask(
    # To maintain logs of the run, you can uncomment the
    # Following lines
    # temporary_directory='./tmp/autoPyTorch_example_tmp_01',
    # output_directory='./tmp/autoPyTorch_example_out_01',
    # delete_tmp_folder_after_terminate=False,
    # delete_output_folder_after_terminate=False,
    seed=42,
)

In [94]:
type(y_train)#.toframe()

pandas.core.frame.DataFrame

In [95]:

X_train.head()

,model,capacity_bytes,pod_slot_num,is_legacy_format,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,smart_3_raw,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_8_normalized,smart_8_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_10_raw,smart_11_normalized,smart_11_raw,smart_12_normalized,smart_12_raw,smart_13_normalized,smart_13_raw,smart_15_normalized,smart_15_raw,smart_16_normalized,smart_16_raw,smart_17_normalized,smart_17_raw,smart_18_normalized,smart_18_raw,smart_22_normalized,smart_22_raw,smart_23_normalized,smart_23_raw,smart_24_normalized,smart_24_raw,smart_27_normalized,smart_27_raw,smart_71_normalized,smart_71_raw,smart_82_normalized,smart_82_raw,smart_90_normalized,smart_90_raw,smart_160_normalized,smart_160_raw,smart_161_normalized,smart_161_raw,smart_163_normalized,smart_163_raw,smart_164_normalized,smart_164_raw,smart_165_normalized,smart_165_raw,smart_166_normalized,smart_166_raw,smart_167_normalized,smart_167_raw,smart_168_normalized,smart_168_raw,smart_169_normalized,smart_169_raw,smart_170_normalized,smart_170_raw,smart_171_normalized,smart_171_raw,smart_172_normalized,smart_172_raw,smart_173_normalized,smart_173_raw,smart_174_normalized,smart_174_raw,smart_175_normalized,smart_175_raw,smart_176_normalized,smart_176_raw,smart_177_normalized,smart_177_raw,smart_178_normalized,smart_178_raw,smart_179_normalized,smart_179_raw,smart_180_normalized,smart_180_raw,smart_181_normalized,smart_181_raw,smart_182_normalized,smart_182_raw,smart_183_normalized,smart_183_raw,smart_184_normalized,smart_184_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_189_normalized,smart_189_raw,smart_190_normalized,smart_190_raw,smart_191_normalized,smart_191_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_195_normalized,smart_195_raw,smart_196_normalized,smart_196_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_200_normalized,smart_200_raw,smart_201_normalized,smart_201_raw,smart_202_normalized,smart_202_raw,smart_206_normalized,smart_206_raw,smart_210_normalized,smart_210_raw,smart_218_normalized,smart_218_raw,smart_220_normalized,smart_220_raw,smart_222_normalized,smart_222_raw,smart_223_normalized,smart_223_raw,smart_224_normalized,smart_224_raw,smart_225_normalized,smart_225_raw,smart_226_normalized,smart_226_raw,smart_230_normalized,smart_230_raw,smart_231_normalized,smart_231_raw,smart_232_normalized,smart_232_raw,smart_233_normalized,smart_233_raw,smart_234_normalized,smart_234_raw,smart_235_normalized,smart_235_raw,smart_240_normalized,smart_240_raw,smart_241_normalized,smart_241_raw,smart_242_normalized,smart_242_raw,smart_244_normalized,smart_244_raw,smart_245_normalized,smart_245_raw,smart_246_normalized,smart_246_raw,smart_247_normalized,smart_247_raw,smart_248_normalized,smart_248_raw,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
serial_number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20W0A0FAF97G,TOSHIBA MG07ACA14TA,14000519643136,34.0,False,100.0,0.0,100.0,0.0,100.0,7840.0,100.0,18.0,100.0,0.0,100.0,0.000000e+00,100.0,0.0,38.0,25176.0,100.0,0.0,NaN,NaN,100.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,0.0,100.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,169104.0,100.0,7.0,100.0,30.0,100.0,39.0,NaN,NaN,100.0,0.0,100.0,0.0,100.0,0.0,200.0,0.0,NaN,NaN,NaN,NaN,NaN,Na

In [96]:
feat_types = []

In [14]:
api.search(
    X_train=X_train,
    y_train=y_train,
    X_test=X_test.copy(),
    y_test=y_test.copy(),
    dataset_name='Trabalho_deep_learning',
    optimize_metric='accuracy',
    total_walltime_limit=300,
    func_eval_time_limit_secs=50
)
# node.get_hyperparameter_search_space(dataset_properties, include=choices_list)

ActiveHyperparameterNotSetError: Hyperparameter is active but has no value set.
SelectRatesClassification:mode, Type: Categorical, Choices: {fpr, fdr, fwe, percentile}, Default: fpr

In [15]:
# help(TabularClassificationTask)